In [38]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import time
import os
import math
import numpy as np
import pyke as pk
import pandas as pd
import k2flix
import matplotlib.pyplot as plt

def collect(oid_list,test):
    tpfs, flux, flares = dict(), dict(), dict()
    for oid in oid_list:

        flares[oid] = pd.read_csv('results/{}/{}_flares.csv'.format(test,oid),
                                 usecols=['istart','istop','istart_no_sys_no_thruster','istop_no_sys_no_thruster'])
        flux[oid] = pd.read_csv('results/{}/{}_flux.csv'.format(test,oid))#,
                                #usecols=['flux_gap','time','flux_model'])
    return flux, flares



cluster = 'M67'
os.chdir('/home/ekaterina/Documents/appaloosa/stars_shortlist/{}'.format(cluster))
oid_list = pd.read_csv('{}_full.txt'.format(cluster),names=['EPIC'])
oid_list = oid_list.EPIC.tolist()
test = 'run_01'
flux, flares = collect(oid_list, test)

- flare number before/after cleaning
- coincidence with flags for cleaned flares
- binomial distribution vs. real distribution


In [39]:
l1, l2 = [], []
for oid in oid_list:

    l1.append(len(flares[oid].istart.dropna()))
    l2.append(len(flares[oid].istart_no_sys_no_thruster.dropna()))

c_flares = pd.DataFrame({'before':l1,'after':l2},index=oid_list)
c_flares['difference'] = (c_flares.after - c_flares.before)/c_flares.before
c_flares.describe()

,after,before,difference
count,75.000000,75.000000,75.000000
mean,20.746667,37.653333,-0.467881
std,16.969841,26.066699,0.124645
min,3.000000,11.000000,-0.880000
25%,11.000000,23.000000,-0.538690
50%,15.000000,28.000000,-0.472973
75%,22.000000,38.000000,-0.381653
max,82.000000,129.000000,-0.250000


In [40]:
flares[oid_list[0]].head()

,istart,istop,istart_no_sys_no_thruster,istop_no_sys_no_thruster
0,153,154,153.0,154.0
1,182,183,182.0,183.0
2,224,225,882.0,882.0
3,548,549,936.0,937.0
4,716,717,1469.0,1470.0


In [41]:
flux[oid_list[0]].head()

,Unnamed: 0,flux_gap,flux_model,time,tpf_flags,tpf_time,isflare,new_isflare,new_tpf_flags,tpf_flags_wo_systematics,isflare_wo_systematics,isflare_no_sys_no_thruster
0,0,333716.362671,333847.941791,2306.611038,524288,2306.611038,0.0,0.0,524288,524288,0.0,False
1,1,333702.796190,333847.028039,2306.631471,0,2306.631471,0.0,0.0,0,0,0.0,False
2,2,333710.525828,333848.275669,2306.651903,0,2306.651903,0.0,0.0,0,0,0.0,False
3,3,333782.384248,333851.483213,2306.672335,0,2306.672335,0.0,0.0,0,0,0.0,False
4,4,333863.244913,333856.449226,2306.692767,0,2306.692767,0.0,0.0,0,0,0.0,False


In [42]:
def flag_df(flux):
    
    '''
    
    Plug in here any measure of the results that contains information for your interpretation.
    
    Input:
    
    flux - DataFrame that contains all previous processing.
    
    Return:
    
    flags - Table with customized inputs
    
    '''
    index = [1,8192,524288]
    columns = ['>',
               '>_after_thruster_removal',
               '>_after_systematics_removal',
               '>_left_overall',
               '>_coinciding_with_flare_candidates',
               '>_coinciding_with_clean_flare_candidates',
               '>_reative_coinciding_with_flare_candidates',
               '>_relative_coinciding_with_clean_flare_candidates',
              ]
    flags = pd.DataFrame(index=index, columns=columns)
    for flag in index:
        flags[columns[0]].loc[flag] = flux[flux.tpf_flags >= flag].shape[0]
        flags[columns[1]].loc[flag] = flux[flux.new_tpf_flags >= flag].shape[0]
        flags[columns[2]].loc[flag] = flux[flux.tpf_flags_wo_systematics >= flag].shape[0]
        flags[columns[3]].loc[flag] = flux[(flux.tpf_flags_wo_systematics >= flag) & (flux.new_tpf_flags >= flag)].shape[0]
        flags[columns[4]].loc[flag] = flux[(flux.isflare == True) & (flux.tpf_flags >= flag)].shape[0]
        flags[columns[5]].loc[flag] = flux[(flux.isflare_no_sys_no_thruster == True) & (flux.tpf_flags >= flag)].shape[0]
        flags[columns[6]].loc[flag] = np.around(flux[(flux.isflare == True) & (flux.tpf_flags >= flag)].shape[0]/flux.shape[0],3)
        flags[columns[7]].loc[flag] = np.around(flux[(flux.isflare_no_sys_no_thruster == True) & (flux.tpf_flags >= flag)].shape[0]/flux.shape[0],3)

    return flags.T

In [43]:
for o in oid_list:
    print('\n',o)
    print(flag_df(flux[o]))


 228682441
                                                  1      8192   524288
>                                                    457    455    201
>_after_thruster_removal                             423    421    185
>_after_systematics_removal                          392    391    167
>_left_overall                                       375    374    158
>_coinciding_with_flare_candidates                    37     37     16
>_coinciding_with_clean_flare_candidates               2      2      0
>_reative_coinciding_with_flare_candidates          0.01   0.01  0.004
>_relative_coinciding_with_clean_flare_candidates  0.001  0.001      0

 211410963
                                                  1      8192   524288
>                                                    275    267    201
>_after_thruster_removal                             268    260    195
>_after_systematics_removal                          235    229    167
>_left_overall                                       

In [ ]:
_